<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO WORKING COLAB LAUNCHER - FIXED FOR REAL
# No llama_cpp_binaries dependency + GPU/CPU auto-detection
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
import shutil
import time
from pathlib import Path

print("🚀 MY-AI-GIZMO WORKING INSTALLER & LAUNCHER")
print("="*70)
print("Fixed: No llama_cpp_binaries dependency + GPU/CPU auto-detection")
print("="*70)

# Paths
DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"
MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

def run_cmd(cmd, check=False, quiet=True, timeout=None):
    """Run command safely"""
    try:
        return subprocess.run(cmd, check=check, capture_output=quiet,
                            text=True, timeout=timeout)
    except:
        return None

def safe_pip_install(packages):
    """Install pip packages"""
    if isinstance(packages, str):
        packages = [packages]
    for pkg in packages:
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                         check=False, timeout=300,
                         stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            pass

# Step 1: Mount Drive
print("\n📂 Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("   ✅ Mounted")
except:
    print("   ⚠️  Mount failed, continuing...")

# Create directories
for d in [DRIVE_BASE, MODELS_DIR, USER_DATA_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Step 2: System packages
print("\n📦 Installing system packages...")
run_cmd(["apt-get", "update", "-qq"])
run_cmd(["apt-get", "install", "-y", "-qq", "build-essential", "cmake", "git", "wget"])
print("   ✅ Done")

# Step 3: Clone repo
print("\n📥 Setting up repository...")
if REPO_DIR.exists():
    os.chdir(REPO_DIR)
    run_cmd(["git", "pull"])
else:
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    os.chdir(REPO_DIR.parent)
    run_cmd(["git", "clone", "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"])

os.chdir(REPO_DIR)
print(f"   ✅ Ready at {REPO_DIR}")

# Step 4: Python packages
print("\n🐍 Installing Python packages...")
safe_pip_install("pip --upgrade")
safe_pip_install(["setuptools", "wheel", "numpy", "requests", "tqdm", "pyyaml"])

# Install llama-cpp-python with server support
print("   Installing llama-cpp-python with server...")
env = dict(os.environ)
env["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=off -DLLAMA_BUILD_SERVER=ON"
subprocess.run([sys.executable, "-m", "pip", "install",
               "llama-cpp-python[server]", "--no-cache-dir", "-q"],
              check=False, env=env, timeout=600)

safe_pip_install(["torch", "transformers", "gradio>=3.50.0", "accelerate",
                 "markdown", "Pillow", "safetensors", "sentencepiece",
                 "protobuf", "flask", "flask-cloudflared"])
print("   ✅ Packages installed")

# Step 5: THE CRITICAL PATCH - Replace llama_cpp_server.py completely
print("\n🔧 Patching llama_cpp_server.py...")

FIXED_LLAMA_SERVER = '''import json
import torch
import os
import pprint
import re
import socket
import subprocess
import sys
import threading
import time
from pathlib import Path
from typing import Any, List
import requests

from modules import shared
from modules.image_utils import (
    convert_image_attachments_to_pil,
    convert_openai_messages_to_images,
    convert_pil_to_base64
)
from modules.logging_colors import logger
from modules.utils import resolve_model_path

llamacpp_valid_cache_types = {"fp16", "q8_0", "q4_0"}

def get_llama_cpp_server_path():
    """Find llama-server executable - NO llama_cpp_binaries dependency"""
    try:
        import llama_cpp
        possible_paths = [
            Path(llama_cpp.__file__).parent / "server" / "llama-server",
            Path(sys.prefix) / "bin" / "llama-server",
            Path("/usr/local/bin/llama-server"),
        ]
        for path in possible_paths:
            if path.exists():
                logger.info(f"Found llama-server at: {path}")
                return str(path)

        # Try system PATH
        result = subprocess.run(['which', 'llama-server'],
                              capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            return result.stdout.strip()

        logger.warning("Using llama-server from PATH")
        return "llama-server"
    except:
        return "llama-server"

class LlamaServer:
    def __init__(self, model_path, server_path=None):
        self.model_path = model_path
        self.server_path = server_path
        self.port = self._find_available_port()
        self.process = None
        self.session = requests.Session()
        self.vocabulary_size = None
        self.bos_token = "~~"
        self.last_prompt_token_count = 0
        self._start_server()

    def encode(self, text, add_bos_token=False, **kwargs):
        if self.bos_token and text.startswith(self.bos_token):
            add_bos_token = False
        url = f"http://127.0.0.1:{self.port}/tokenize"
        response = self.session.post(url, json={"content": text, "add_special": add_bos_token})
        return response.json().get("tokens", [])

    def decode(self, token_ids, **kwargs):
        url = f"http://127.0.0.1:{self.port}/detokenize"
        response = self.session.post(url, json={"tokens": token_ids})
        return response.json().get("content", "")

    def prepare_payload(self, state):
        payload = {
            "temperature": state["temperature"] if not state["dynamic_temperature"] else (state["dynatemp_low"] + state["dynatemp_high"]) / 2,
            "top_k": state["top_k"],
            "top_p": state["top_p"],
            "min_p": state["min_p"],
            "repeat_penalty": state["repetition_penalty"],
            "seed": state["seed"],
        }
        return payload

    def _process_images_for_generation(self, state: dict) -> List[Any]:
        pil_images = []
        if 'image_attachments' in state and state['image_attachments']:
            pil_images.extend(convert_image_attachments_to_pil(state['image_attachments']))
        return pil_images

    def is_multimodal(self) -> bool:
        return shared.args.mmproj not in [None, 'None']

    def generate_with_streaming(self, prompt, state):
        url = f"http://127.0.0.1:{self.port}/completion"
        payload = self.prepare_payload(state)

        token_ids = self.encode(prompt, add_bos_token=state.get("add_bos_token", False))
        self.last_prompt_token_count = len(token_ids)
        payload["prompt"] = token_ids

        max_new_tokens = state.get('max_new_tokens', 200)
        payload.update({"n_predict": max_new_tokens, "stream": True, "cache_prompt": True})

        response = self.session.post(url, json=payload, stream=True)
        full_text = ""

        try:
            for line in response.iter_lines():
                if not line:
                    continue
                try:
                    line = line.decode('utf-8')
                    if line.startswith('data: '):
                        line = line[6:]
                    data = json.loads(line)
                    if data.get('content'):
                        full_text += data['content']
                        yield full_text
                    if data.get('stop'):
                        break
                except:
                    continue
        finally:
            response.close()

    def generate(self, prompt, state):
        output = ""
        for output in self.generate_with_streaming(prompt, state):
            pass
        return output

    def get_logits(self, prompt, state, n_probs=128, use_samplers=False):
        url = f"http://127.0.0.1:{self.port}/completion"
        payload = self.prepare_payload(state)
        payload.update({
            "prompt": self.encode(prompt),
            "n_predict": 0,
            "logprobs": True,
            "n_probs": n_probs,
            "stream": False,
        })
        response = self.session.post(url, json=payload)
        result = response.json()
        if "completion_probabilities" in result:
            return result["completion_probabilities"][0]["top_probs" if use_samplers else "top_logprobs"]
        raise Exception("Unexpected response format")

    def _get_vocabulary_size(self):
        url = f"http://127.0.0.1:{self.port}/v1/models"
        response = self.session.get(url).json()
        if "data" in response and len(response["data"]) > 0:
            model_info = response["data"][0]
            if "meta" in model_info and "n_vocab" in model_info["meta"]:
                self.vocabulary_size = model_info["meta"]["n_vocab"]

    def _get_bos_token(self):
        url = f"http://127.0.0.1:{self.port}/props"
        response = self.session.get(url).json()
        if "bos_token" in response:
            self.bos_token = response["bos_token"]

    def _find_available_port(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('', 0))
            return s.getsockname()[1]

    def _start_server(self):
        if self.server_path is None:
            self.server_path = get_llama_cpp_server_path()

        # GPU AUTO-DETECTION
        has_gpu = torch.cuda.is_available()
        gpu_layers = shared.args.gpu_layers if has_gpu else 0

        if has_gpu:
            logger.info(f"GPU detected: {torch.cuda.get_device_name(0)}")
        else:
            logger.warning("No GPU → CPU mode")

        cmd = [
            self.server_path,
            "--model", self.model_path,
            "--ctx-size", str(shared.args.ctx_size),
            "--batch-size", str(shared.args.batch_size),
            "--port", str(self.port),
            "--no-webui",
        ]

        # GPU flags ONLY if GPU exists
        if has_gpu and gpu_layers > 0:
            cmd += ["--gpu-layers", str(gpu_layers)]

        if shared.args.threads > 0:
            cmd += ["--threads", str(shared.args.threads)]

        env = os.environ.copy()

        logger.info(f"llama.cpp | GPU={has_gpu} | layers={gpu_layers} | ctx={shared.args.ctx_size}")

        self.process = subprocess.Popen(cmd, stderr=subprocess.PIPE, bufsize=0, env=env)

        threading.Thread(target=self._log_stderr, daemon=True).start()

        # Wait for health
        health_url = f"http://127.0.0.1:{self.port}/health"
        for _ in range(60):
            if self.process.poll() is not None:
                raise RuntimeError(f"Server exited with code {self.process.returncode}")
            try:
                if self.session.get(health_url, timeout=1).status_code == 200:
                    break
            except:
                pass
            time.sleep(1)

        self._get_vocabulary_size()
        self._get_bos_token()
        return self.port

    def _log_stderr(self):
        try:
            for line in iter(self.process.stderr.readline, b''):
                print(line.decode('utf-8', errors='replace').strip(), file=sys.stderr)
        except:
            pass

    def stop(self):
        if self.process:
            self.process.terminate()
            try:
                self.process.wait(timeout=5)
            except:
                self.process.kill()
            self.process = None

    def __del__(self):
        self.stop()
'''

llama_server_file = REPO_DIR / "modules" / "llama_cpp_server.py"
llama_server_file.write_text(FIXED_LLAMA_SERVER, encoding='utf-8')
print("   ✅ llama_cpp_server.py completely replaced (NO llama_cpp_binaries)")

# Step 6: Link user_data
print("\n🔗 Linking user_data...")
local_user_data = REPO_DIR / "user_data"
if local_user_data.exists() and not local_user_data.is_symlink():
    shutil.rmtree(local_user_data)
if not local_user_data.exists():
    local_user_data.symlink_to(USER_DATA_DIR)
(USER_DATA_DIR / "logs").mkdir(parents=True, exist_ok=True)
print("   ✅ Linked")

# Step 7: Download model
print(f"\n⬇️  Checking model...")
if MODEL_FILE.exists():
    print(f"   ✅ Model exists ({MODEL_FILE.stat().st_size/(1024**3):.2f} GB)")
else:
    print("   📥 Downloading model (3.8 GB, one-time)...")
    MODEL_FILE.parent.mkdir(parents=True, exist_ok=True)
    run_cmd(["wget", "-q", "--show-progress", "-O", str(MODEL_FILE),
            "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"],
           timeout=900)
    if MODEL_FILE.exists():
        print(f"   ✅ Downloaded ({MODEL_FILE.stat().st_size/(1024**3):.2f} GB)")

# Step 8: Link models
print("\n🔗 Linking models...")
repo_models = REPO_DIR / "models"
if not repo_models.is_symlink():
    if repo_models.exists():
        shutil.rmtree(repo_models)
    repo_models.symlink_to(MODELS_DIR)
print("   ✅ Linked")

# Step 9: Detect GPU and build command
print("\n🖥️  Detecting hardware...")
has_gpu = False
try:
    import torch
    has_gpu = torch.cuda.is_available()
    if has_gpu:
        print(f"   ✅ GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("   ℹ️  No GPU, using CPU")
except:
    print("   ℹ️  Using CPU")

cmd = [sys.executable, "server.py"]

if has_gpu:
    cmd.extend(["--gpu-layers", "35"])
else:
    cmd.extend(["--cpu"])

cmd.extend([
    "--threads", "4",
    "--listen",
    "--listen-host", "0.0.0.0",
    "--share",
    "--model", str(MODEL_FILE),
    "--loader", "llama.cpp"
])

print(f"\n📝 Configuration:")
print(f"   Hardware: {'GPU' if has_gpu else 'CPU'}")
print(f"   Model: {MODEL_FILE.name}")
print(f"   Command: {' '.join(str(x) for x in cmd[:8])}...")

# Step 10: LAUNCH!
print("\n" + "="*70)
print("🚀 LAUNCHING SERVER - WATCH FOR GRADIO LINK BELOW!")
print("="*70 + "\n")

os.chdir(REPO_DIR)

try:
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                              universal_newlines=True, bufsize=1)

    found_link = False
    for line in process.stdout:
        print(line, end='')

        if ('gradio.live' in line.lower() or
            ('running on' in line.lower() and 'http' in line)) and not found_link:
            print("\n" + "🎉"*35)
            print("✅ SERVER IS RUNNING! CLICK THE LINK ABOVE ☝️")
            print("🎉"*35)
            print(f"\n💡 Tips:")
            print(f"   • Click the Gradio link")
            print(f"   • Hardware: {'GPU' if has_gpu else 'CPU'}")
            print(f"   • Chats save to: {USER_DATA_DIR / 'logs'}")
            print(f"   • Press Ctrl+C to stop\n")
            found_link = True

    process.wait()

except KeyboardInterrupt:
    print("\n⏹️  Stopped by user")
except Exception as e:
    print(f"\n⚠️  Error: {e}")

print("\n" + "="*70)
print("Done!")
print("="*70)